In [1]:
!pip install pandas==1.4.3 scikit-learn fastparquet numpy

In [2]:
#import polars as pl
import pandas as pd

In [3]:
_path = "/content/drive/MyDrive/MLOpsZoomcamp2023/homework/01-intro/"

df_jan = pd.read_parquet("yellow_tripdata_2022-01.parquet")

# **Q1**

R/ Q1: 19

# **Q2**

In [4]:
df_jan['duration'] = df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']
df_jan['duration_min']= (df_jan['duration']/ pd.Timedelta(minutes=1)).astype('int16')

In [5]:
df_jan['duration_min'].std()

46.44541509327847

R/ Q2: 46.45

# **Q3**

In [6]:
df_jan = df_jan[(df_jan['duration_min'] >= 1) & (df_jan['duration_min'] <= 60)]


In [7]:
number_filtered = 2423325/2463931
print(number_filtered)

0.9835198307095451


R/ Q3: 0.98

# **Q4**

Converting into dict

In [8]:
#df_jan_dict = df_jan.to_dict()
df_jan_dict = df_jan[['PULocationID', 'DOLocationID']]
df_jan_dict = df_jan_dict.astype(str).to_dict(orient='records')

In [9]:
len(df_jan_dict[0])

2

Fit a dictionary vectorizer

In [10]:
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer()
features = vec.fit_transform(df_jan_dict)
# features

Get feature matrix

*R/ Q4:515*

# *Q5*

In [11]:
target_ = df_jan[['duration_min']].copy()
target_ = target_.to_dict(orient='records')
target = vec.transform(target_)
target

<2423325x515 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [12]:
import numpy as np
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(features, df_jan['duration_min'].values)
#reg = LinearRegression().fit(features, target)
reg

LinearRegression()

Save and load model

In [13]:
from joblib import dump, load

dump(reg, 'model2.joblib') 

['model2.joblib']

In [14]:
reg = load('model2.joblib') 

In [15]:
from sklearn.metrics import mean_squared_error

y_pred = reg.predict(features)
y_train = features

y_train

<2423325x515 sparse matrix of type '<class 'numpy.float64'>'
	with 4846650 stored elements in Compressed Sparse Row format>

Error

In [16]:
error = mean_squared_error(df_jan['duration_min'].values, y_pred, squared=False)
print(error)

7.016720374381209


*R/ Q5: 6.99*

# *Q6*

### Validation dataset

In [17]:
df_feb = pd.read_parquet("yellow_tripdata_2022-02.parquet")
df_feb['duration'] = df_feb['tpep_dropoff_datetime'] - df_feb['tpep_pickup_datetime']
df_feb['duration_min']= (df_feb['duration']/ pd.Timedelta(minutes=1)).astype('int16')
df_feb = df_feb[(df_feb['duration_min'] >= 1) & (df_feb['duration_min'] <= 60)]


df_feb_dict = df_feb[['PULocationID', 'DOLocationID']]
df_feb_dict = df_feb_dict.to_dict(orient='records')
features_val = vec.transform(df_feb_dict)


y_pred_val = reg.predict(features_val)

error_val = mean_squared_error(df_feb['duration_min'].values, y_pred_val, squared=False)
print(error_val)

14.80287201767579


R/ Q6:14.8